In [1]:
import dask
from dask import dataframe as dd
import pyarrow as pa

In [131]:
N_PARTITIONS = 10
OUTPUT_LOCATION = "/tmp/lake/big_mimic"

tables_to_fetch = {
    #"noteevents": ["row_id", "category"],
    "patients": ["gender"],
    "admissions": ["admission_type"],
} 

In [132]:
%%time
dfs = {
    table: dd.read_sql_table(table, DB_URI, schema="mimiciii", columns=columns, index_col="subject_id", npartitions=N_PARTITIONS)
    for table, columns in tables_to_fetch.items()
}

CPU times: user 78.7 ms, sys: 6.98 ms, total: 85.7 ms
Wall time: 1.6 s


In [133]:
%%time
pat_df = dfs["patients"]
fhir_genders = {"F": "female", "M": "male"}
pat_df["clean_gender"] = pat_df["gender"].map(lambda x: fhir_genders.get(x, "unknown"))

CPU times: user 4.17 ms, sys: 123 µs, total: 4.3 ms
Wall time: 4.22 ms


In [134]:
%%time
#df.to_parquet("./lake/big_mimic", schema={"icd9_code": pa.string()})
for table, df in dfs.items():
    df.to_parquet(f"{OUTPUT_LOCATION}/{table}")

CPU times: user 475 ms, sys: 107 ms, total: 583 ms
Wall time: 3.31 s


# Read and join

In [120]:
patients_read = dd.read_parquet(f"{OUTPUT_LOCATION}/patients", filters=[("subject_id", "<", 249)])
admissions_read = dd.read_parquet(f"{OUTPUT_LOCATION}/admissions")
print("nb patients: \t", len(patients_read))
print("nb admissions: \t", len(admissions_read))

nb patients: 	 9462
nb admissions: 	 58976


In [125]:
print("===== PATIENTS =====")
print(patients_read.head())
print("===== ADMISSIONS =====")
print(admissions_read.head())

===== PATIENTS =====
           gender  subject_id__1 clean_gender
subject_id                                   
249             F            249       female
250             F            250       female
251             M            251         male
252             M            252         male
253             F            253       female
===== ADMISSIONS =====
           admission_type
subject_id               
22              EMERGENCY
23               ELECTIVE
23              EMERGENCY
24              EMERGENCY
25              EMERGENCY


In [126]:
joined = patients_read.merge(admissions_read)#, left_on="gender", right_on="admission_type")
print(joined.head())
print("Total joined: \t", len(joined))

           gender  subject_id__1 clean_gender admission_type
subject_id                                                  
2               M              2         male        NEWBORN
3               M              3         male      EMERGENCY
4               F              4       female      EMERGENCY
5               M              5         male        NEWBORN
6               F              6       female       ELECTIVE
Total joined: 	 12252


In [102]:
import os
os.getcwd()

'/Users/jason/arkhn/poc-deltalake/src/dask'